In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random as rd
from scipy.optimize import fsolve

%load_ext autoreload
%autoreload 2
from generate_homophilic_graph_asymmetric import homophilic_barabasi_albert_graph_assym

In [2]:
#################################
# This function is useful if
# you have an empirical in-group edges
# and want to calculate numerically
# the values of homophily.
################################

def numeric_solver(p):
    h_aa,h_bb,ca,beta_a,beta_b = p
    
    fb = 1 - minority_fraction
    fa = minority_fraction

    M = maj_maj + maj_min + min_min
    m_bb = maj_maj
    m_ab = maj_min
    m_aa = min_min

    pbb = float(m_bb)/(m_aa+m_bb+m_ab)
    paa = float(m_aa)/(m_aa+m_bb+m_ab)
    pba = float(m_ab)/(m_aa+m_bb+m_ab)
    pab = pba


    h_ab = 1- h_aa
    h_ba = 1- h_bb

    A = (h_aa - h_ab)*(h_ba - h_bb)
    B = ((2*h_bb - (1-fa) * h_ba)*(h_aa - h_ab) + (2*h_ab - fa*(2*h_aa - h_ab))*(h_ba - h_bb))
    C = (2*h_bb*(2*h_ab - fa*(2*h_aa - h_ab)) - 2*fa*h_ab*(h_ba - h_bb) - 2*(1-fa)*h_ba * h_ab)
    D = - 4*fa*h_ab*h_bb
    
    P = [A, B, C, D]

    Z = fa / (1 - beta_a)
    K = fb / (1 - beta_b)

    #p_aa = (fa * h_aa *ba)/( h_aa *ba + h_ab *bb) # this is the exact result

    #p_bb = (fb * h_bb *bb)/( h_bb *bb + h_ba *ba) # this is the exact result
    
    #p_ab = (fa*h_ab*bb) /(h_ab*bb +h_aa*ba) + (fb*h_ba*ba)/(h_ba*ba +h_bb*bb)
    
    #P_aa_analytic = float(p_aa)/(p_aa + p_bb + p_ab) # this is the exact result
    #P_bb_analytic = float(p_bb)/(p_aa + p_bb + p_ab) # this is the exact result



    return ( (pbb* ((fa * h_aa * Z)+ (fb*(1-h_bb)*Z) + (fa*(1-h_aa)*K)+(fb *h_bb*K) )) - (fb * h_bb * K ) , 
            
            (paa* ((fa * h_aa * Z)+ (fb*(1-h_bb)*Z) + (fa*(1-h_aa)*K)+(fb *h_bb*K) ))  - (fa * h_aa * Z ) ,
            
            beta_a - float(fa*h_aa)/ (h_aa*ca + h_ab * (2 - ca)) - float(fb*h_ba)/(h_ba*ca + h_bb*(2-ca))  ,
            
            beta_b - float(fb*h_bb)/ (h_ba*ca + h_bb * (2 - ca)) - float(fa*h_ab)/(h_aa*ca + h_ab*(2-ca)) , 
            
            ca - np.roots(P)[root_num] )
    


In [3]:
def calculate_edges_empirical(G):
    
    maj_maj = 0
    maj_min = 0
    min_min = 0
    # blue: majority (b) ; red:minority (a)
    for e in G.edges():
        n1,n2 = e
        g1 = G.node[n1]['color']
        g2 = G.node[n2]['color']
        
        if g1 == g2:
            if g1 == 'blue':
                maj_maj += 1
            else:
                min_min += 1
        else:
            maj_min += 1
        
    #print 'min_min , maj_min , maj_maj' , min_min , maj_min , maj_maj
          
    return min_min , maj_min , maj_maj
    

In [4]:
def measuring_empirical_homophily(maj_maj,maj_min,min_min,minority_fraction):
    
    '''
    the estimation is largely depends on the initial values that are fed to the fsolver.
    therefore it is important to initialize the fsolve functions with realist values to begin with.
    Note that the exponent of the degree distribution (e) is related to the degree growth (beta)
    as follows: e = float(1)/beta + 1. Therefore, if one can estimate the exponent of the 
    degree distribution for the minority and majority, it will help to initialze the values for beta.
    
    maj-maj etc are total number of majority to majority links. etc

    '''

    global root_num

    for root_num in [0,1,2]:
        
        # h_aa,h_bb,ca,beta_a,beta_b
        # these are initializations that you need to tune
        
        h_aa_anal,h_bb_anal,ca,beta_a,beta_b = fsolve(numeric_solver,(1,1,0.5,0.5,0.5)) 
        
        print('estimations',h_aa_anal,h_bb_anal,ca, beta_a , beta_b)
        e_min,e_maj = float(1)/beta_a + 1, float(1)/beta_b + 1
        
        if 0<ca and ca <2: #this is acceptable range for ca

            return h_aa_anal, h_bb_anal , e_min , e_maj 
    

### Example

In [5]:
N = 2000
m = 2
h_ab = 0. # haa = 0.7
h_ba = 0. # hbb = 0.7
minority_fraction = 0.2

G = homophilic_barabasi_albert_graph_assym(N , m , minority_fraction, h_ab , h_ba)
min_min , maj_min , maj_maj = calculate_edges_empirical(G)
h_aa_prediction, h_bb_prediction , e_min , e_maj = measuring_empirical_homophily(maj_maj,maj_min,min_min,minority_fraction)

#### values from the network function
1.0 1.0
estimations 1.6142020155010215 0.5330277634392193 10.264488025495526 0.9967761245418599 0.9957596705925416
estimations 1.429377849566564 0.809738176392699 0.7213857199358904 0.7227558093394747 0.3743226456885339


/ssd/lespin/virtualenv/python3.5/lib/python3.5/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [6]:
print('h_aa_prediction',h_aa_prediction,'h_bb_prediction', h_bb_prediction)

h_aa_prediction 1.429377849566564 h_bb_prediction 0.809738176392699


### Notes

[1] To ensure that the estimates are accurate, you can run maximum-likelihood estimate over all the values of homophiles and check which values give the closer estimates to paa and pbb, where pbb = float(m_bb)/(m_aa+m_bb+m_ab).

[2] Another solution is to give beta_a and beta_b as inputs instead of initial guess. In this way, the numerical solver will have easier time to predict the results since it has fewer parameters to predict. I haven't made the code yet.

<div style="text-align:center;"><h1>Relational Classification</h1></div>

In [9]:
import sys
import networkx as nx
from collections import Counter

sys.path.append('../code')
from org.gesis.network.network import Network

def load_graph(fn):
    return nx.read_gpickle(fn)

def get_minority_fraction(G):
    tmp = Counter([G.graph['group'][G.graph['labels'].index(G.node[n][G.graph['class']])] for n in G.nodes()])
    return tmp['m'] / float(tmp['m']+tmp['M'])

def get_edge_type_counts(G):
    tmp = Counter(['{}{}'.format( G.graph['group'][G.graph['labels'].index(G.node[e[0]][G.graph['class']])], G.graph['group'][G.graph['labels'].index(G.node[e[1]][G.graph['class']])] ) for e in G.edges()])
    min_min , maj_min , min_maj, maj_maj = tmp['mm'], tmp['Mm'] , tmp['mM'], tmp['MM']
    return min_min , maj_min , min_maj, maj_maj

def get_homophily_old_code(graph, smooth=1):

    fm = get_minority_fraction(graph)
    Emm, EMm, EmM, EMM = get_edge_type_counts(graph)

    Emm += smooth
    EMM += smooth
    EMm += smooth
    EmM += smooth

    fM = 1 - fm
    emm = float( Emm / (Emm + EMm + EmM + EMM) )
    eMM = float( EMM / (Emm + EMm + EmM + EMM) )
    
    hmm = float(-2 * emm * fm * fM) / ((emm * (fm ** 2)) - (2 * emm * fm * fM) + (emm * (fM ** 2) - (fm ** 2)))
    hMM = float(-2 * eMM * fM * fm) / ((eMM * (fM ** 2)) - (2 * eMM * fM * fm) + (eMM * (fm ** 2) - (fM ** 2)))

    return hmm, hMM


def estimate_homophily_empirical(graph, fm=None, EMM=None, EMm=None, EmM=None, Emm=None, gammaM_in=None, gammam_in=None, verbose=False):
    
    hmm = []
    hMM = []
    diff = []

    if graph is not None and (fm is None or EMM is None or EMm is None or EmM is None or Emm is None):
        Emm, EMm, EmM, EMM = get_edge_type_counts(graph)
        fm = get_minority_fraction(graph)
    elif graph is None and (fm is None or EMM is None or EMm is None or EmM is None or Emm is None):
        raise Exception('Missing important parameters.')

    E = EMM + EMm + EmM + Emm
    min_min = Emm / E
    maj_maj = EMM / E
    min_maj = (EmM + EMm) / E
    maj_min = (EmM + EMm) / E
    fM = 1 - fm

    # calculating ca for directed
    K_m = min_min + maj_min
    K_M = maj_maj + min_maj
    K_all = K_m + K_M

    cm = (K_m) / K_all
    if verbose:
        print(cm)

    for h_mm_ in np.arange(0, 1.01, 0.01):
        for h_MM_ in np.arange(0, 1.01, 0.01):

            h_mm_analytical = h_mm_
            h_MM_analytical = h_MM_

            h_mM_analytical = 1 - h_mm_analytical
            h_Mm_analytical = 1 - h_MM_analytical

            if gammaM_in is None:
                try:
                    gamma_M = float(fM * h_MM_analytical) / ((h_Mm_analytical * cm) + (h_MM_analytical * (2 - cm))) + 
                              float(fm * h_mM_analytical) / ((h_mm_analytical * cm) + (h_mM_analytical * (2 - cm)))
                except RuntimeWarning:
                    if verbose:
                        print('break 2')
                    break
            else:
                gamma_M = gammaM_in


            if gammam_in is None:
                try:
                    gamma_m = float(fm * h_mm_analytical) / ((h_mm_analytical * cm) + (h_mM_analytical * (2 - cm))) + 
                              float(fM * h_Mm_analytical) / ((h_Mm_analytical * cm) + (h_MM_analytical * (2 - cm)))
                except RuntimeWarning:
                    if verbose:
                        print('break 1')
                    break
            else:
                gamma_m = gammam_in


            K = 1 - gamma_m
            Z = 1 - gamma_M

            if ((fm * h_mm_analytical * Z) + ((1 - fm) * (1 - h_mm_analytical) * K) == 0 or (fM * h_MM_analytical * K) + (fm * (1 - h_MM_analytical) * Z)) == 0:
                break

            pmm_analytical = float(fm * h_mm_analytical * Z) / ((fm * h_mm_analytical * Z) + ((1 - fm) * (1 - h_mm_analytical) * K))
            pMM_analytical = float(fM * h_MM_analytical * K) / ((fM * h_MM_analytical * K) + (fm * (1 - h_MM_analytical) * Z))

            if min_min + min_maj + maj_maj == 0:
                # bipartite
                raise NotImplementedError('This model does not support bipartite networks.')
            else:
                pmm_emp = float(min_min) / (min_min + min_maj)
                pMM_emp = float(maj_maj) / (maj_maj + maj_min)

            _diff = abs(pmm_emp - pmm_analytical) + abs(pMM_emp - pMM_analytical)
            diff.append(_diff)
            hmm.append(h_mm_analytical)
            hMM.append(h_MM_analytical)

            if verbose and _diff < 0.02:
                print()
                print('pmm_emp', pmm_emp, 'pmm_analytical', pmm_analytical)
                print('pMM_emp', pMM_emp, 'pMM_analytical', pMM_analytical)
                print('pMM_diff', abs(pmm_emp - pmm_analytical), 'pMM_diff', abs(pMM_emp - pMM_analytical) , 'diff' ,  _diff)
                print('h_mm_analytical', h_mm_analytical, 'h_MM_analytical', h_MM_analytical, 'cm_analytical', cm)

    best = np.argmin(diff)
    return hMM[best], hmm[best]


<h2>Empirical Networks</h2>

In [10]:
### Loading network
G = load_graph('../data/Caltech36.gpickle')

### computing minority fraction
min_min , maj_min , min_maj, maj_maj = get_edge_type_counts(G)
minority_fraction = get_minority_fraction(G)
print(min_min , maj_min + min_maj, maj_maj)
print(minority_fraction)
print()

### computing homophily MLE (Fariba's code: 2020-04-22, 5:01 PM)
h_aa_prediction, h_bb_prediction , e_min , e_maj = measuring_empirical_homophily(maj_maj,maj_min+min_maj,min_min,minority_fraction)
print('h_mm_prediction',h_aa_prediction,'h_MM_prediction', h_bb_prediction, 'average: ', np.mean([h_aa_prediction,h_bb_prediction]))
print()

### computing homophily estimation (Fariba's code: 2019-10-14, 1:49 PM)
hm,hM = get_homophily_old_code(G)
h = np.mean([hm,hM])
print('hmm ',hm,'hMM', hM, 'h_avg', h)

### computing homophily code for directed networks
hMM,hmm = estimate_homophily_empirical(G)
h = np.mean([hmm,hMM])
print('hmm ',hmm,'hMM', hMM, 'h_avg', h)

### Expected (from results submitted to ICWSM):
# hmm 0.59
# hMM 0.47
# h   0.56

2351 6846 6267
0.3252496433666191

estimations 0.8717116172528777 0.5225589893609123 15.682269479085036 0.9969119165394742 0.9970111244449505
estimations 0.7996384950832048 0.6045738354567473 0.7408737001794895 0.5609917813390208 0.46411225249673044
h_mm_prediction 0.7996384950832048 h_MM_prediction 0.6045738354567473 average:  0.702106165269976

hmm  0.7652598337554604 hMM 0.43831243100092065 h_avg 0.6017861323781906


/ssd/lespin/virtualenv/python3.5/lib/python3.5/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


hmm  0.35000000000000003 hMM 0.37 h_avg 0.36


In [16]:
### Loading network
G = load_graph('../data/Escorts.gpickle')

### computing minority fraction
min_min , maj_min , min_maj, maj_maj = get_edge_type_counts(G)
minority_fraction = get_minority_fraction(G)
print(min_min , maj_min + min_maj, maj_maj)
print(minority_fraction)
print()

### computing homophily MLE (Fariba's code: 2020-04-22, 5:01 PM)
h_aa_prediction, h_bb_prediction , e_min , e_maj = measuring_empirical_homophily(maj_maj,maj_min+min_maj,min_min,minority_fraction)
print('h_mm_prediction',h_aa_prediction,'h_MM_prediction', h_bb_prediction, 'average: ', np.mean([h_aa_prediction,h_bb_prediction]))
print()

### computing homophily estimation (Fariba's code: 2019-10-14, 1:49 PM)
hm,hM = get_homophily_old_code(G)
h = np.mean([hm,hM])
print('hmm ',hm,'hMM', hM, 'h_avg', h)

### computing homophily code for directed networks
hMM,hmm = estimate_homophily_empirical(G)
h = np.mean([hmm,hMM])
print('hmm ',hmm,'hMM', hMM, 'h_avg', h)

### Expected:
# hmm 0.0
# hMM 0.0
# h   0.0

0 39044 0
0.39593544530783026

estimations 0.6844546164104003 0.13273584637611838 1.3210542221835888 0.5365596039440803 0.18939032631881936
h_mm_prediction 0.6844546164104003 h_MM_prediction 0.13273584637611838 average:  0.4085952313932593



/ssd/lespin/virtualenv/python3.5/lib/python3.5/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


hmm  7.814356933193873e-05 hMM 3.3571716826090884e-05 h_avg 5.585764307901481e-05
hmm  0.0 hMM 0.0 h_avg 0.0


In [17]:
### Loading network
G = load_graph('../data/Swarthmore42.gpickle')

### computing minority fraction
min_min , maj_min , min_maj, maj_maj = get_edge_type_counts(G)
minority_fraction = get_minority_fraction(G)
print(min_min , maj_min + min_maj, maj_maj)
print(minority_fraction)
print()

### computing homophily MLE (Fariba's code: 2020-04-22, 5:01 PM)
h_aa_prediction, h_bb_prediction , e_min , e_maj = measuring_empirical_homophily(maj_maj,maj_min+min_maj,min_min,minority_fraction)
print('h_mm_prediction',h_aa_prediction,'h_MM_prediction', h_bb_prediction, 'average: ', np.mean([h_aa_prediction,h_bb_prediction]))
print()

### computing homophily estimation (Fariba's code: 2019-10-14, 1:49 PM)
hm,hM = get_homophily_old_code(G)
h = np.mean([hm,hM])
print('hmm ',hm,'hMM', hM, 'h_avg', h)

### computing homophily code for directed networks
hMM,hmm = estimate_homophily_empirical(G)
h = np.mean([hmm,hMM])
print('hmm ',hmm,'hMM', hMM, 'h_avg', h)

### Expected (from results submitted to ICWSM):
# hmm 0.53
# hMM 0.53
# h   0.53

13689 25069 14968
0.4924292297564187

estimations 0.8399962951586148 0.5222384946530384 17.670305437019255 0.9907578470078275 0.9944170669110332
estimations 0.7011971177850213 0.7039269542804274 0.9811443540581335 0.4964128363609678 0.5031387724402454
h_mm_prediction 0.7011971177850213 h_MM_prediction 0.7039269542804274 average:  0.7025620360327243



/ssd/lespin/virtualenv/python3.5/lib/python3.5/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/ssd/lespin/virtualenv/python3.5/lib/python3.5/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


hmm  0.5253805830077487 hMM 0.5407055778616089 h_avg 0.5330430804346789
hmm  0.35000000000000003 hMM 0.37 h_avg 0.36


In [18]:
### Loading network
G = load_graph('../data/USF512009.gpickle')

### computing minority fraction
min_min , maj_min , min_maj, maj_maj = get_edge_type_counts(G)
minority_fraction = get_minority_fraction(G)
print(min_min , maj_min + min_maj, maj_maj)
print(minority_fraction)
print()

### computing homophily MLE (Fariba's code: 2020-04-22, 5:01 PM)
h_aa_prediction, h_bb_prediction , e_min , e_maj = measuring_empirical_homophily(maj_maj,maj_min+min_maj,min_min,minority_fraction)
print('h_mm_prediction',h_aa_prediction,'h_MM_prediction', h_bb_prediction, 'average: ', np.mean([h_aa_prediction,h_bb_prediction]))
print()

### computing homophily estimation (Fariba's code: 2019-10-14, 1:49 PM)
hm,hM = get_homophily_old_code(G)
h = np.mean([hm,hM])
print('hmm ',hm,'hMM', hM, 'h_avg', h)

### computing homophily code for directed networks
hMM,hmm = estimate_homophily_empirical(G)
h = np.mean([hmm,hMM])
print('hmm ',hmm,'hMM', hMM, 'h_avg', h)

### Expected (from results submitted to ICWSM):
# hmm 0.39
# hMM 0.45
# h   0.45

1798 8528 5271
0.40410557184750734

estimations 0.6330283491590013 0.525339514201356 14.639405394766781 0.9916335507247485 0.9948500847449296
estimations 0.7298584914929618 0.6860597171769255 0.8241639501574772 0.4628485822968569 0.5081503962443819
h_mm_prediction 0.7298584914929618 h_MM_prediction 0.6860597171769255 average:  0.7079591043349436



/ssd/lespin/virtualenv/python3.5/lib/python3.5/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/ssd/lespin/virtualenv/python3.5/lib/python3.5/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


hmm  0.3491504266836932 hMM 0.4749560745170153 h_avg 0.41205325060035425
hmm  0.21 hMM 0.33 h_avg 0.27


In [19]:
### Loading network
G = load_graph('../data/Wikipedia.gpickle')

### computing minority fraction
min_min , maj_min , min_maj, maj_maj = get_edge_type_counts(G)
minority_fraction = get_minority_fraction(G)
print(min_min , maj_min + min_maj, maj_maj)
print(minority_fraction)
print()

### computing homophily MLE (Fariba's code: 2020-04-22, 5:01 PM)
h_aa_prediction, h_bb_prediction , e_min , e_maj = measuring_empirical_homophily(maj_maj,maj_min+min_maj,min_min,minority_fraction)
print('h_mm_prediction',h_aa_prediction,'h_MM_prediction', h_bb_prediction, 'average: ', np.mean([h_aa_prediction,h_bb_prediction]))
print()

### computing homophily estimation (Fariba's code: 2019-10-14, 1:49 PM)
hm,hM = get_homophily_old_code(G)
h = np.mean([hm,hM])
print('hmm ',hm,'hMM', hM, 'h_avg', h)

### computing homophily code for directed networks
hMM,hmm = estimate_homophily_empirical(G)
h = np.mean([hmm,hMM])
print('hmm ',hmm,'hMM', hMM, 'h_avg', h)

### Expected (from results submitted to ICWSM):
# hmm 0.70
# hMM 0.60
# h   0.60

125 584 2434
0.15337711069418386

estimations 1.019075560770563 0.546961899568215 7.344023058188957 0.997784106765801 0.9981404867052277
estimations 0.9271866657631672 0.6730628160221609 0.34132533483820254 0.5506424662098885 0.4895786936977739
h_mm_prediction 0.9271866657631672 h_MM_prediction 0.6730628160221609 average:  0.800124740892664

hmm  2.4279760725082284 hMM 0.5826041849704131 h_avg 1.5052901287393208


/ssd/lespin/virtualenv/python3.5/lib/python3.5/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


hmm  0.5 hMM 0.46 h_avg 0.48
